# Evaluation criteria

The goal of this assignment is to get a view on your hands-on "data engineering" skills.  
At our company, our data scientists and engineers collaborate on projects.  
Your main focus will be creating performant & robust data flows.  
For a take-home-assignment, we cannot grant you access to our infrastructure.  
The assignement below measures your proficiency in general programming, data science & engineering tasks using python.  
Completion should not take more than half a day.

**We expect you to be proficient in:**
 * SQL queries (Sybase IQ system)
 * ETL flows (In collaboration with existing teams)
 * General python to glue it all together
 * Python data science ecosystem (Pandas + SKlearn)
 
**In this exercise we expect you to demonstrate your ability to / knowledge of:**
 * Building a data science runtime
 * PEP8 / Google python styleguide
 * Efficiently getting the job done
 * Choose meaningfull names for variables & functions
 * Writing maintainable code (yes, you might need to document some steps)
 * Help a data scientist present interactive results.
 * Offer predictions via REST api

# Setting-up a data science workspace

We allow you full freedom in setting up a data science runtime.  
The main objective is having a runtime where you can run this notebook and the code you will develop.  
You can choose for a local setup on your pc, or even a cloud setup if you're up for it.   

**In your environment, you will need things for:**
 * https request
 * python3 (not python2 !!)
 * (geo)pandas
 * interactive maps (e.g. folium, altair, ...)
 * REST apis
 
**Deliverables we expect**:
 * notebook with the completed assignment
 * list of packages for your runtime (e.g. yml or txt file)
 * evidence of a working API endpoint

# Importing packages

We would like you to put all your import statements here, together in 1 place.  
Before submitting, please make sure you remove any unused imports :-)  

In [1]:
## your imports go here.  You get pandas for free.

import pandas as pd
import requests
import json  # check if needed
import unittest

# required for importing a saved sklearn model
import joblib
import lightgbm as lgb

# Data ingestion exercises

## Getting store location data from an API

**Goal:** Obtain a pandas dataframe  
**Hint:** You will need to normalise/flatten the json, because it contains multiple levels  
**API call:** https://ecgplacesmw.colruytgroup.com/ecgplacesmw/v3/nl/places/filter/clp-places  

In [28]:
provided_url = "https://ecgplacesmw.colruytgroup.com/ecgplacesmw/v3/nl/places/filter/clp-places"

def get_clp_places(url):
    try:
        request_result = requests.get(url, timeout = 5)
        request_result.raise_for_status()
    except requests.exceptions.HTTPError as err:
        raise(err)
    except requests.exceptions.RequestException as e:
        # catastrophic error. bail.
        raise(e)
    
    request_content = request_result.json()
    request_content_pd = pd.json_normalize(request_content)
    
    return request_content_pd
    

df_clp = get_clp_places(provided_url)
df_clp.head(10)
#print(df_clp[["placeId","commercialName","geoCoordinates.latitude"]])


,placeId,commercialName,branchId,sourceStatus,sellingPartners,handoverServices,moreInfoUrl,routeUrl,isActive,placeSearchOpeningHours,...,placeType.id,placeType.longName,placeType.placeTypeDescription,geoCoordinates.latitude,geoCoordinates.longitude,address.streetName,address.houseNumber,address.postalcode,address.cityName,address.countryName
0,902,AALST (COLRUYT),4156,AC,"[QUALITY, 3RDPARTY]","[CSOP_ORDERABLE, PREPAID_PARCEL]",https://www.colruyt.be/nl/colruyt-openingsuren...,"https://maps.apple.com/?daddr=50.933074,4.0538972",True,"[{'date': '27-05-2022', 'opens': 830, 'closes'...",...,1,Winkel,Winkel,50.933074,4.053897,BRUSSELSE STEENWEG,41,9300,AALST,België
1,946,AALTER (COLRUYT),4218,AC,"[QUALITY, 3RDPARTY]","[CSOP_ORDERABLE, PREPAID_PARCEL]",https://www.colruyt.be/nl/colruyt-openingsuren...,"https://maps.apple.com/?daddr=51.0784761,3.450...",True,"[{'date': '27-05-2022', 'opens': 830, 'closes'...",...,1,Winkel,Winkel,51.078476,3.450013,LOSTRAAT,66,9880,AALTER,België
2,950,AARSCHOT (COLRUYT),4222,AC,"[QUALITY, 3RDPARTY]","[CSOP_ORDERABLE, PREPAID_PARCEL]",https://www.colruyt.be/nl/colruyt-openingsuren...,"https://maps.apple.com/?daddr=50.9760369,4.811...",True,"[{'date': '27-05-2022', 'opens': 830, 'closes'...",...,1,Winkel,Winkel,50.976037,4.811097,LEUVENSESTEENWEG,241,3200,AARSCHOT,België
3,886,ALSEMBERG (COLRUYT),4138,AC,"[QUALITY, 3RDPARTY]","[CSOP_ORDERABLE, PREPAID_PARCEL]",https://www.colruyt.be/nl/colruyt-openingsuren...,"https://maps.apple.com/?daddr=50.7415212,4.336719",True,"[{'date': '27-05-2022', 'opens': 830, 'closes'...",...,1,Winkel,Winkel,50.741521,4.336719,BRUSSELSESTEENWEG,19,1652,ALSEMBERG,België
4,783,AMAY (COLRUYT),3853,AC,"[QUALITY, 3RDPARTY]","[CSOP_ORDERABLE, PREPAID_PARCEL]",https://www.colruyt.be/nl/colruyt-openingsuren...,"https://maps.apple.com/?daddr=50.5599284,5.306...",True,"[{'date': '27-05-2022', 'opens': 830, 'closes'...",...,1,Winkel,Winkel,50.559928,5.306195,CHAUSSEE DE TONGRES,247,4540,AMAY,België
5,650,ANDENNE (COLRUYT),3596,AC,"[QUALITY, 3RDPARTY]","[CSOP_ORDERABLE, PREPAID_PARCEL]",https://www.colruyt.be/nl/colruyt-openingsuren...,"https://maps.apple.com/?daddr=50.4917055,5.093...",True,"[{'date': '27-05-2022', 'opens': 830, 'closes'...",...,1,Winkel,Winkel,50.491706,5.093003,RUE DE LA PAPETERIE,4,5300,ANDENNE,België
6,669,ANDERLECHT (HERBETTELN) COLR,3620,AC,"[QUALITY, 3RDPARTY]","[CSOP_ORDERABLE, PREPAID_PARCEL]",https://www.colruyt.be/nl/colruyt-openingsuren...,"https://maps.apple.com/?daddr=50.8439965,4.309...",True,"[{'date': '27-05-2022', 'opens': 830, 'closes'...",...,1,Winkel,Winkel,50.843997,4.309948,MAURICE HERBETTELAAN,57,1070,ANDERLECHT,België
7,744,ANDERLECHT (VEEWEYDE) COLRUYT,3759,AC,"[QUALITY, 3RDPARTY]","[CSOP_ORDERABLE, PREPAID_PARCEL]",https://www.colruyt.be/nl/colruyt-openingsuren...,"https://maps.apple.com/?daddr=50.8275378372,4....",True,"[{'date': '27-05-2022', 'opens': 830, 'closes'...",...,1,Winkel,Winkel,50.827538,4.302574,BERGENSESTEENWEG,824,1070,ANDERLECHT,België
8,448,ANDERLUES (COLRUYT),3074,AC,"[QUALITY, 3RDPARTY]","[CSOP_ORDERABLE, PREPAID_PARCEL]",https://www.colruyt.be/nl/colruyt-openingsuren...,"https://maps.apple.com/?daddr=50.401257,4.2797751",True,"[{'date': '27-05-2022', 'opens': 830, 'closes'...",...,1,Winkel,Winkel,50.401257,4.279775,RUE DE LA STATION,4,6150,ANDERLUES,België
9,681,ANS (COLRUYT),3644,AC,"[QUALITY, 3RDPARTY]","[CSOP_ORDERABLE, PREPAID_PARCEL]",https://www.colruyt.be/nl/colruyt-openingsuren...,"https://maps.apple.com/?daddr=50.6588119,5.532...",True,"[{'date': '27-05-2022', 'opens': 830, 'closes'...",...,1,Winkel,Winkel,50.658812,5.532497,RUE DES FRANCAIS,88,4430,ANS,België


### Quality checks

We would like you to add several checks on this data based on these constraints:  
 * records > 200
 * latitude between 49 and 52
 * longitude between 2 and 7
 
We dont want you to create a full blown test suite here, we're just gonna use 'asserts' from unittest

In [29]:
pd.options.mode.chained_assignment = None  # default='warn'
# cfr: https://stackoverflow.com/questions/20625582/how-to-deal-with-settingwithcopywarning-in-pandas

tc = unittest.TestCase('__init__')

# test on number of entries in dataframe
tc.assertGreater(len(df_clp.index), 200, "Less then 200 CLP entries in dataset")

# test on latitude being between values
df_throw_away = df_clp["geoCoordinates.latitude"].apply(lambda lat: tc.assertTrue( 49.0 < lat < 52.0, str(lat) + ": latitude value is outside 49.0-52.0"))

# alternative way to test longitude being between values with better error message
df_clp_test = df_clp[["placeId","commercialName","geoCoordinates.latitude","geoCoordinates.longitude"]]
df_clp_test.rename(columns={'geoCoordinates.latitude':'latitude','geoCoordinates.longitude':'longitude'}, inplace=True)
#df_throw_away = df_clp_test.apply(lambda row: print(row), axis = 1)
df_throw_away = df_clp_test.apply(lambda row: tc.assertTrue( 2 < row['longitude'] < 7.0,
                                                            "longitude value is: " + str(row['longitude']) +
                                                            " for " + str(row['placeId']) + " - " + str(row['commercialName']) +
                                                            "is outside range 2.0 - 7.0"
                                                            ),
                                  axis=1)
df_throw_away = {}

### Feature creation

Create a new column "antwerpen" which is 1 for all stores in Antwerpen (province) and 0 for all others 

In [90]:
df_clp["antwerpen"] = df_clp.apply(lambda x: 1 if (2000 <= int(x['address.postalcode']) < 3000) else 0, axis = 1)

df_clp["antwerpen"].value_counts()

0    217
1     35
Name: antwerpen, dtype: int64

## Predict used car value

A datascientist in our team made a basic model to predict car prices.  
The model was saved to disk ('lgbr_cars.model') using joblib's dump fuctionality.  
Documentation states the model is a LightGBM Regressor, trained using the sk-learn api.  

**As engineer, your task it to expose this model as REST-api.** 

First, retrieve the model via the function below.  
Change the path according to your setup.  

In [8]:
tc = unittest.TestCase('__init__')

def retrieve_model(path):
    try:
        file_handle = open(path,'rb')
    except IOError as e:
        raise(e)
    trained_model = joblib.load(file_handle)
    return trained_model

lgbr_cars = retrieve_model("./lgbr_cars.model")

tc.assertEqual(str(type(lgbr_cars)),"<class 'lightgbm.sklearn.LGBMRegressor'>", type(lgbr_cars))

Now you have your trained model, lets do a functional test based on the parameters below.  
You have to present the parameters in this order.  

* vehicleType: coupe
* gearbox: manuell
* powerPS: 190
* model: NaN
* kilometer: 125000
* monthOfRegistration: 5 
* fuelType: diesel
* brand: audi

Based on these parameters, you should get a predicted value of 14026.35068804
However, the model doesnt accept string inputs, see the integer encoding below:

In [9]:
model_test_input = [[3,1,190,-1,125000,5,3,1]]

In [17]:
def make_prediction(trained_model, single_input) -> float:
    predicted_value = trained_model.predict(single_input)

    # predicted_value is of type numpy.ndarray with a single element
    # get the float out
    predicted_value_float = predicted_value.item()
    return predicted_value_float

predicted_value = make_prediction(lgbr_cars, model_test_input)

tc.assertAlmostEqual(predicted_value, 14026.35, places=2)

Now you got this model up and running, we want you to **expose it as a rest api.**  
We don't expect you to set up any authentication.  
We're not looking for beautiful inputs, just make it work.  
**Building this endpoint should NOT be done in a notebook, but in proper .py file(s)**

Once its up and running, use it to predict the following input:
* [-1,1,0,118,150000,0,1,38] ==> prediction should be 13920.70

## Geospatial data exercise
The goal of this exercise is to read in some data from a shape file and visualize it on a map
- The map should be dynamic. I want to zoom in and out to see more interesting aspects of the map
- We want you to visualize the statistical sectors within a distance of 2KM of your home location.

Specific steps to take:
- Read in the shape file
- Transform to WGS coordinates
- Create a distance function (Haversine)
- Create variables for home_lat, home_lon and perimeter_distance
- Calculate centroid for each nis district
- Calculate the distance to home for each nis district centroid 
- Figure out which nis districts are near your home
- Create dynamic zoomable map
- Visualize the nis districts near you (centroid <2km away), on the map


In [100]:
# Some imports to help you along the way
import geopandas as gpd
import folium # you can use any viz library you prefer
# import shapefile   
# import gdal
from json import dumps
import random

In [101]:
# part 1: Reading in the data
# get this file from https://statbel.fgov.be/sites/default/files/files/opendata/Statistische%20sectoren/sh_statbel_statistical_sectors_20200101.shp.zip 
df = gpd.read_file('./data/external/shapefiles/AD_0_StatisticSector.shp')

# needed to set crs before converting projection
df.set_crs(epsg=31370, inplace=True)

df = df.to_crs({'init': 'epsg:4326'}) # change projection to wgs84 

df.head(5)


/home/me/github/data-engineering-exercise/py3_dsclp/lib/python3.6/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


,geometry
0,"POLYGON Z ((4.39247 51.13717 0.00000, 4.39519 ..."
1,"POLYGON Z ((4.39374 51.13858 0.00000, 4.39356 ..."
2,"POLYGON Z ((4.37785 51.13689 0.00000, 4.37634 ..."
3,"POLYGON Z ((4.39140 51.14451 0.00000, 4.39057 ..."
4,"POLYGON Z ((4.39374 51.13858 0.00000, 4.39350 ..."


In [102]:
# One of the data scientists discovered stackoverflow ;-) and copypasted something from https://gis.stackexchange.com/questions/166820/geopandas-return-lat-and-long-of-a-centroid-point
# A data science engineer should be able to speed this next code up

df[['centroid_lon', 'centroid_lat']]=df.apply(lambda row: [row.geometry.centroid.x, row.geometry.centroid.y],axis=1,
                             result_type='expand')
df.head(3)
# for i in range(0, len(df)):
#     df.loc[i,'centroid_lon'] = df.geometry.centroid.x.iloc[i]
#     df.loc[i,'centroid_lat'] = df.geometry.centroid.y.iloc[i]

,geometry,centroid_lon,centroid_lat
0,"POLYGON Z ((4.39247 51.13717 0.00000, 4.39519 ...",4.388756,51.132421
1,"POLYGON Z ((4.39374 51.13858 0.00000, 4.39356 ...",4.384630,51.138929
2,"POLYGON Z ((4.37785 51.13689 0.00000, 4.37634 ...",4.374922,51.139997


In [103]:
# Let's create some variables to indicate the location of your interest 
# president Kenedy park
home_lat = 50.800632
home_lon = 3.273495
perimeter_distance = 2 # km

In [104]:
# At some point we will need a distance function (google the Haversine formula, and implement it)
# https://stackoverflow.com/questions/4913349/haversine-formula-in-python-bearing-and-distance-between-two-gps-points

from math import radians, cos, sin, asin, sqrt

def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance in kilometers between two points 
    on the earth (specified in decimal degrees)
    """
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles. Determines return value units.

    # convert decimal degrees to radians 
    try:
        lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
        # haversine formula 
        dlon = lon2 - lon1 
        dlat = lat2 - lat1 
        a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
        c = 2 * asin(sqrt(a)) 
    except Exception as e:
        c = 0

    return c * r


Next, implement some sanity checks for your distance function 

In [105]:
# implement sanity checks here
tc = unittest.TestCase('__init__')

# determined 2 coordinates on google maps
# 1: crossing N43 and N36: lon=3.325061, lat=50.863578
# 2: crossing N43 and N382: lon=3.398745, lat=50.898231 
# this is almost a straight line for which google gives a distance of 6.4 km

distance_km = haversine(3.325061, 50.863578, 3.398745, 50.898231 )

tc.assertAlmostEqual(distance_km, 6.4, places=1)

In [106]:
# calculate distance from centroid NIS district to President Kenedy location

df['dist_to_pres_ken_km']=df.apply(lambda row: haversine( home_lon, home_lat, row['centroid_lon'], row['centroid_lat']),axis=1)
df.head(3)

,geometry,centroid_lon,centroid_lat,dist_to_pres_ken_km
0,"POLYGON Z ((4.39247 51.13717 0.00000, 4.39519 ...",4.388756,51.132421,86.373487
1,"POLYGON Z ((4.39374 51.13858 0.00000, 4.39356 ...",4.384630,51.138929,86.419929
2,"POLYGON Z ((4.37785 51.13689 0.00000, 4.37634 ...",4.374922,51.139997,85.859799


In [107]:
# mark NIS districts within perimeter_distance

df['within_perimeter'] = df.apply(lambda row: 1 if (row['dist_to_pres_ken_km'] < perimeter_distance) else 0, axis = 1)
df['within_perimeter'].value_counts()

0    19779
1       15
Name: within_perimeter, dtype: int64

Now, create a dynamical map 

In [ ]:
https://github.com/akkana/scripts/blob/master/mapping/polidistmap.py
https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoDataFrame.to_json.html

In [108]:
def random_html_color():
    r = random.randint(0,256)
    g = random.randint(0,256)
    b = random.randint(0,256)
    return '#%02x%02x%02x' % (r, g, b)


def style_fcn(x):
    '''The style function can key off x['properties']['NAME10']
        which will be strings like 'Senate District 42'
        but for now, let's just return random colors.
    '''
    return { 'fillColor': random_html_color() }


In [109]:
# select only the shapes within the perimeter to visualize

df_map = df.loc[df['within_perimeter'] == 1]
df_map_within_perimeter = df_map['geometry']

# convert the shape info to geo_json for visualisation
geo_json_string = df_map_within_perimeter.to_json()

df_map_within_perimeter.head(20)


6134    POLYGON Z ((3.28424 50.82066 0.00000, 3.28454 ...
6143    POLYGON Z ((3.27603 50.81815 0.00000, 3.27642 ...
6144    POLYGON Z ((3.27040 50.81666 0.00000, 3.27060 ...
6148    POLYGON Z ((3.26345 50.81558 0.00000, 3.26526 ...
6161    POLYGON Z ((3.29301 50.80228 0.00000, 3.29253 ...
6162    POLYGON Z ((3.28646 50.80720 0.00000, 3.28682 ...
6163    POLYGON Z ((3.29328 50.81009 0.00000, 3.29323 ...
6164    POLYGON Z ((3.29401 50.80959 0.00000, 3.29407 ...
6166    POLYGON Z ((3.26935 50.79793 0.00000, 3.26931 ...
6167    POLYGON Z ((3.28172 50.80127 0.00000, 3.28230 ...
6169    POLYGON Z ((3.26935 50.79793 0.00000, 3.26935 ...
6204    POLYGON Z ((3.26309 50.80687 0.00000, 3.26367 ...
6205    MULTIPOLYGON Z (((3.25030 50.81207 0.00000, 3....
6207    POLYGON Z ((3.26403 50.79977 0.00000, 3.26326 ...
6222    POLYGON Z ((3.26948 50.79047 0.00000, 3.26975 ...
Name: geometry, dtype: geometry

In [110]:
# create layer with selected shapes in geo_json format
geo_json_nid_districts = folium.GeoJson(geo_json_string, style_function=style_fcn,
                        highlight_function=highlight_fcn)

# create the folium map
my_map = folium.Map(location=[home_lat, home_lon], zoom_start=12)

geo_json_nid_districts.add_to(my_map)

folium.LayerControl().add_to(my_map)

my_map